In [2]:
!pip install requests census pandas us

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 7.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from census import Census
from us import states
from google.colab import userdata

### Pull in PA demographic data off ACS API

In [4]:
# 1. Load your API key
key = userdata.get("CensusAPIKey")
c = Census(key)

# 2. Choose the relevant dataset (e.g., acs5 for 5-year estimates)
dataset = c.acs5

# 3. Define variable codes to pull in and readable column names to replace them with in DF
variables = (
    'NAME',                 # Geography Name
    'B01001_001E',          # Total Population
    'B01001A_001E',         # Race: White Alone
    'B01001B_001E',         # Race: Black/African American Alone
    'B03002_012E',          # Ethnicity: Hispanic or Latino
    'B19013_001E',          # Income: Median Household Income
    'B15003_022E',          # Education: Bachelor's Degree or Higher (Pop 25+)
    'B01001_020E',          # Age: 65-66 years (often higher turnout)
    'B01001_021E',          # Age: 67-69 years
    'B01001_022E',          # Age: 70-74 years
    'B01001_023E',          # Age: 75-79 years
    'B01001_024E',          # Age: 80-84 years
    'B01001_025E',          # Age: 85+ years
    'B01001_007E',          # Age: 18-19 years (often lower turnout)
    'B01001_008E',          # Age: 20 years
    'B01001_009E',          # Age: 21 years
    'B01001_010E',          # Age: 22-24 years
    'B25003_002E',          # Housing: Owner-Occupied Units
    'B25003_003E',          # Housing: Renter-Occupied Units
    'B05001_006E'           # Citizenship: Foreign Born (Naturalized/Not)
)

labels = (
    'NAME',
    'Total Population',
    'Pop White Alone',
    'Pop Black Alone',
    'Pop Hispanic or Latino',
    'Median Household Income',
    'Edu Bachelors or Higher',
    'Age 65-66',
    'Age 67-69',
    'Age 70-74',
    'Age 75-79',
    'Age 80-84',
    'Age 85 Plus',
    'Age 18-19',
    'Age 20',
    'Age 21',
    'Age 22-24',
    'Owner Occupied Units',
    'Renter Occupied Units',
    'Foreign Born Pop'
)

# 4. Define geographic aggregations and criterias
# Example: Get data for all counties in a specific state (e.g., Colorado)
geo_for = 'county:*'
geo_in_PA = f'state:{states.PA.fips}' # Using the 'us' library to get FIPS code for PA
geo_in_MI = f'state:{states.MI.fips}' # Using the 'us' library to get FIPS code for MI
geo_in_NC = f'state:{states.NC.fips}' # Using the 'us' library to get FIPS code for NC

# 5. Run the query and store as a pandas DataFrame
# Pull in 2024 data for all three states
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_PA}, year=2024)
df_pa24 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_MI}, year=2024)
df_mi24 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_NC}, year=2024)
df_nc24 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
# Pull in 2020 data for all three states
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_PA}, year=2020)
df_pa20 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_MI}, year=2020)
df_mi20 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_NC}, year=2020)
df_nc20 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
# Pull in 2016 data for all three states
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_PA}, year=2016)
df_pa16 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_MI}, year=2016)
df_mi16 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
r = dataset.get(variables, {'for': geo_for, 'in': geo_in_NC}, year=2016)
df_nc16 = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
# Add year column
df_pa24['year'] = 2024
df_mi24['year'] = 2024
df_nc24['year'] = 2024
df_pa20['year'] = 2020
df_mi20['year'] = 2020
df_nc20['year'] = 2020
df_pa16['year'] = 2016
df_mi16['year'] = 2016
df_nc16['year'] = 2016
# Merge DFs
df_fullData = pd.concat([df_pa16, df_mi16, df_nc16, df_pa20, df_mi20, df_nc20, df_pa24, df_mi24, df_nc24], ignore_index=True)
# Print the first few rows of the DataFrame
print(df_fullData)
has_nulls = df_fullData.isnull().values.sum()
print(f"The DataFrame has {has_nulls} null values.")

                                 NAME  Total Population  Pop White Alone  \
0           Pike County, Pennsylvania           56210.0          50416.0   
1         Snyder County, Pennsylvania           40246.0          38891.0   
2    Susquehanna County, Pennsylvania           41832.0          40887.0   
3       Crawford County, Pennsylvania           87027.0          83395.0   
4           Erie County, Pennsylvania          279133.0         243823.0   
..                                ...               ...              ...   
745      Wayne County, North Carolina          118652.0          60942.0   
746     Wilkes County, North Carolina           65935.0          57339.0   
747     Wilson County, North Carolina           79290.0          36124.0   
748     Yadkin County, North Carolina           37574.0          31232.0   
749     Yancey County, North Carolina           18797.0          17315.0   

     Pop Black Alone  Pop Hispanic or Latino  Median Household Income  \
0             

In [5]:
df_fullData.describe()

,Total Population,Pop White Alone,Pop Black Alone,Pop Hispanic or Latino,Median Household Income,Edu Bachelors or Higher,Age 65-66,Age 67-69,Age 70-74,Age 75-79,Age 80-84,Age 85 Plus,Age 18-19,Age 20,Age 21,Age 22-24,Owner Occupied Units,Renter Occupied Units,Foreign Born Pop,year
count,7.500000e+02,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,1.328216e+05,98362.241333,19762.445333,10170.068000,55289.121333,17788.726667,1529.801333,2024.240000,2660.294667,1768.976000,1114.241333,980.261333,1848.533333,965.694667,935.686667,2653.885333,36202.316000,16317.528000,4993.280000,2020.000000
std,2.342732e+05,149980.499051,67623.683045,23856.634729,14150.560753,37981.625589,2427.976504,3130.770157,4047.341596,2648.978917,1712.555864,1642.044722,3176.530119,1647.234806,1610.716257,4782.819684,59616.915519,34756.347068,13922.526958,3.268166
min,2.102000e+03,1962.000000,0.000000,11.000000,30013.000000,134.000000,27.000000,58.000000,58.000000,31.000000,13.000000,0.000000,4.000000,0.000000,0.000000,1.000000,905.000000,106.000000,2.000000,2016.000000
25%,2.561125e+04,22369.500000,407.250000,640.000000,45124.250000,2430.750000,397.750000,525.250000,741.500000,515.500000,307.500000,225.500000,300.250000,145.000000,130.750000,422.250000,8724.750000,2329.250000,215.750000,2016.000000
50%,5.479400e+04,44130.000000,2423.500000,2438.500000,52612.000000,4899.000000,717.000000,961.500000,1300.500000,866.000000,532.000000,416.000000,694.500000,356.500000,352.000000,980.500000,15603.500000,5650.000000,842.000000,2020.000000
75%,1.376472e+05,109671.000000,14316.000000,8391.000000,62925.250000,14285.500000,1633.750000,2104.500000,2819.500000,1874.500000,1185.250000,975.000000,1820.000000,993.500000,918.250000,2564.500000,37183.250000,14694.500000,2626.750000,2024.000000
max,1.772259e+06,992002.000000,694872.000000,245882.000000,127208.000000,271137.000000,20479.000000,24968.000000,35827.000000,22590.000000,12683.000000,12416.000000,23726.000000,14043.000000,13652.000000,37451.000000,453652.000000,327523.000000,123286.000000,2024.000000
